In [1]:
import librosa
import numpy as np
import os
import cv2
import pickle
from keras.utils import np_utils

def get_labels(data_dir):
    labels = []
    files = os.listdir(data_dir)
    files.sort()
    for file in files:
        catname_full, _ = os.path.splitext(file)
        catname = catname_full.split('_')[-1]
        labels.append(catname)
    print(labels)
    return labels

def load_data(data_dir): #From Directory
    MAX_NUM = 1000
    x_load = []
    y_load = []
    labels = get_labels(data_dir)
    dirs = labels
    for cat in dirs: #load directory
        files_dir = data_dir + cat 
        files = os.listdir(files_dir)
        for file in files[:MAX_NUM]:
            file_path = os.path.join(files_dir,file)
            mfccs = get_mfcc(file_path) # shape (20 , 32)
            x = np.array(mfccs).astype('float32')
            x_load.append(x)
            y_load.append(labels.index(cat))  # directory name as label
    return x_load,y_load

def dump_picle(features, labels):
    features = np.array(features).astype('float32')
    labels = np.array(labels).astype('float32')
    print(features.shape)
    print(labels.shape)
    features=features.reshape(features.shape[0]*features.shape[1], features.shape[2])
    labels=labels.reshape(labels.shape[0]*labels.shape[1], labels.shape[2])
    print(features.shape)
    print(labels.shape)
    with open("features", "wb") as f:
        pickle.dump(features, f, protocol=4)
    with open("labels", "wb") as f:
        pickle.dump(labels, f, protocol=4)

def loadFromPickle():
    with open("features", "rb") as f:
        features = np.array(pickle.load(f))
    with open("labels", "rb") as f:
        labels = np.array(pickle.load(f))
    return features, labels
def dump_label_name(dirs):
    with open("label_names", "wb") as f:
        pickle.dump(dirs, f, protocol=4)    
def load_label_name():
    with open("label_names", "rb") as f:
        dirs = np.array(pickle.load(f))
    return dirs
def prepress_labels(labels):
    labels = np_utils.to_categorical(labels) # one-hot编码 把类别id转换为表示当前类别的向量，比如0 1 2 =》 [[1 0 0] [0 1 0] [0 0 1]]
    return labels

def write_image(cat,index,array):
    dest_dir = os.path.join(to_dir,cat)
    if os.path.exists(dest_dir)==False:
        os.makedirs(dest_dir)
    filename = cat + "_" +str(index) + ".png"
    path = os.path.join(dest_dir,filename)
    cv2.imwrite( path , array)

def load_npy_data(data_dir):
    x_load = []
    y_load = []
    MAX_NUM = 100
    labels = get_labels(data_dir)
    dirs = labels
    write_img_file = False
    files = os.listdir(data_dir)
    for file in files[:]:
        catname_full,_ = os.path.splitext(file)
        catname = catname_full.split('_')[-1]
        file = os.path.join(data_dir,file) 
        cat = os.path.basename(file)
        imgs = np.load(file)
        print(imgs.shape)
        # print(np.shape(imgs)) # (133572, 784)
        # imgs = imgs.astype('float32') / 255.
        imgs = imgs[:MAX_NUM, :]
        x_load.append(imgs)
        y = [labels.index(catname) for _ in range(MAX_NUM)]
        y = np.array(y).astype('float32')
        y = y.reshape(y.shape[0], 1)
        y_load.append(y)
        if write_img_file:
            for index,img in enumerate(imgs):
                img = img.reshape(28,28)
                write_image(catname,index,img)
    return x_load,y_load

/usr/local/lib/python3.5/dist-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/usr/local/lib/python3.5/dist-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/usr/local/lib/python3.5/dist-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/usr/local/lib/python3.5/dist-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use

/usr/local/lib/python3.5/dist-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/usr/local/lib/python3.5/dist-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/usr/local/lib/python3.5/dist-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/usr/local/lib/python3.5/dist-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use

In [2]:
!ls

Untitled.ipynb	features  labels  npy_data  static


In [8]:
# thanks to https://github.com/akshaybahadur21/QuickDraw
import h5py
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.layers import Dense,Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout
from keras.utils import np_utils, print_summary
import tensorflow as tf
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import pickle
import keras
from keras.callbacks import TensorBoard

from data_utils import loadFromPickle,prepress_labels,get_labels

def keras_model1(input_shape,num_classes):
    # 构建模型
    model = Sequential()
    model.add(Dense(512, activation='relu',input_shape=input_shape))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # [编译模型] 配置模型，损失函数采用交叉熵，优化采用Adadelta，将识别准确率作为模型评估
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
    filepath = "model1.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    return model, callbacks_list

def keras_model(input_shape,num_classes):
    model = Sequential()
    model.add(Conv2D(8, (5, 5), input_shape=input_shape, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
    model.add(Conv2D(8, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    model.add(Flatten())
    #model.add(Dense(512, activation='relu'))
    #model.add(Dropout(0.6))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    filepath = "model.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    return model, callbacks_list

def test_model():
    features, labels = loadFromPickle()
    labels_count = len(load_label_name())
    # features, labels = augmentData(features, labels)
    features, labels = shuffle(features, labels)
    labels=prepress_labels(labels)
    train_x, test_x, train_y, test_y = train_test_split(features, labels, random_state=0,test_size=0.2)
    train_x = train_x.reshape(train_x.shape[0], 28, 28, 1)
    test_x = test_x.reshape(test_x.shape[0], 28, 28, 1)
    model, callbacks_list = keras_model( (28,28,1,) , labels_count ) 
    print_summary(model)
    model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=10, batch_size=128)
    # 开始评估模型效果 # verbose=0为不输出日志信息
    score = model.evaluate(test_x, test_y, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1]) # 准确度       
    model.save('model.h5')

def test_model1():
    features, labels = loadFromPickle()
    labels_count = len(load_label_name())
    # features, labels = augmentData(features, labels)
    features, labels = shuffle(features, labels)
    labels=prepress_labels(labels)
    train_x, test_x, train_y, test_y = train_test_split(features, labels, random_state=0,test_size=0.1)
    # train_x = train_x.reshape(train_x.shape[0], 28, 28, 1)
    # test_x = test_x.reshape(test_x.shape[0], 28, 28, 1)
    model, callbacks_list = keras_model1( (28*28,) , labels_count ) 
    print_summary(model)
    model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=20, batch_size=128)

    # 开始评估模型效果 # verbose=0为不输出日志信息
    score = model.evaluate(test_x, test_y, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1]) # 准确度    

    model.save('model1.h5')

test_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 24, 24, 8)         208       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 12, 12, 8)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 8, 8, 8)           1608      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 4, 4, 8)           0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
__________

In [7]:
from flask import Flask
from flask import request,Response
from flask import render_template
from keras.models import load_model
import tensorflow as tf
import base64
import numpy as np
import cv2
import os
import json

import sys
root_dir = "/xiaobai/"
sys.path.append(root_dir)
from xiaobai import XiaoBai,BaseSkill
keyword_model = root_dir+'resources/小白.pmdl'
xiaobai = XiaoBai(keyword_model=keyword_model)

from data_utils import get_labels

app = Flask(__name__)

graph = tf.get_default_graph()
# model = load_model('model1.h5') # 加载训练模型
model = load_model('model.h5') # 加载训练模型
label_names =  load_label_name().tolist()

@app.route('/labels')
def get_labels():
    # return json.dumps(label_names)
    return Response(json.dumps(label_names), mimetype='application/json')

@app.route('/',methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        # print(request.form)
        img_b64encode = request.form.get("base64img","")
        # img_b64encode = "data:image/jpeg;base64,/9j/4AAQS......."
        img_b64decode = base64.b64decode(img_b64encode[23:])  # base64解码
        img_array = np.fromstring(img_b64decode,np.uint8) # 转换np序列
        # img_array = np.fromstring(img_b64decode,np.float32) # 转换np序列
        print(img_array.shape)
        img = cv2.imdecode(img_array,cv2.COLOR_BGR2RGB)  # 转换Opencv格式
        img = cv2.resize(img, (28, 28))
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        print(gray.shape)
        with graph.as_default():
            pred_probab = keras_predict(model, gray)
            pred_class = list(pred_probab[0]).index(max(pred_probab[0]))
            print("may be " , label_names[pred_class] , " probab: " ,  pred_probab[0])
            formated = list( map(lambda x,i : (x.item(),label_names[i]) , pred_probab[0],[i for i in range(len(label_names))]) )
            xiaobai.speak("我猜你画的是"+label_names[pred_class])
            return json.dumps(sorted(formated,reverse=True))
    else :
        # return "hello"
        return app.send_static_file('index.html')

def keras_predict(model, image):
    processed = keras_process_image(image)
    # print("processed: " + str(processed.shape))
    pred_probab = model.predict(processed)
    return pred_probab

def keras_process_image(img):
    image_x = 28
    image_y = 28
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1,image_x, image_y,1))    
    #img = np.reshape(img, (-1 , image_x*image_y))
    return img

if __name__ == '__main__':  # pragma: no cover
    app.run(host="0.0.0.0",port=80)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


(2245,)
(28, 28)
may be  rainbow  probab:  [0.00e+00 1.00e+00 0.00e+00 0.00e+00 1.53e-43]
小白：我猜你画的是rainbow
(2032,)
(28, 28)
may be  umbrella  probab:  [0. 0. 0. 0. 1.]
小白：我猜你画的是umbrella


In [7]:
label_names.tolist()

['book', 'rainbow', 'sun', 't-shirt', 'umbrella']